In [1]:
! pip install selenium

  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.6.15-py3-none-any.whl.metadata (2.4 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 10.1 MB/

In [11]:
! pip install webdriver_manager

  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.2-cp313-cp313-macosx_10_13_universal2.whl.metadata (35 kB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.2-cp313-cp313-macosx_10_13_universal2.whl (199 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [webdriver_manager]


In [44]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import datetime


In [115]:
url = "https://www.boamp.fr/pages/entreprise-accueil/"
today = datetime.date.today().strftime("%d/%m/%Y")

In [116]:
options = Options()
options.add_argument("--start-maximized")  # Run in headless mode
options.binary_location ="/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

time.sleep(3)
driver.find_element(By.XPATH, './/a[@class="fr-btn"]').click()
time.sleep(3)
driver.find_element(By.XPATH, './/button[@class="fr-accordion__btn ng-binding"]').click()
time.sleep(5)
driver.find_element(By.XPATH, './/div[@class="odswidget-timerange__from"]/input').send_keys(today)
time.sleep(3)
driver.find_element(By.XPATH, './/div[@class="odswidget-timerange__to"]/input').send_keys(today)
time.sleep(5)
driver.find_element(By.XPATH, './/a[@class="fr-btn"]').click()
time.sleep(3)



In [117]:
nb_pages = len(driver.find_elements(By.XPATH, './/div[@class="fr-container no-print"]/nav/ul/li'))

In [119]:
nb_pages

6

In [142]:
liens=[]
for i in range(1,nb_pages+1):
    print(i)
    driver.find_element(By.XPATH, './/div[@class="fr-container no-print"]/nav/ul/li[' + str(i) + ']').click()
    time.sleep(5)
    liens.extend([lien.get_attribute('href') for lien in driver.find_elements(By.XPATH, './/a[@class="fr-btn fr-my-1w ng-binding"]')]) # les liens


1
2
3
4
5
6


In [152]:
# driver.get(liens[0])  # aller sur le premier lien
driver.get(driver.find_element(By.XPATH, './/a[@class="fr-btn fr-my-1w margin-left-auto-sm"]').get_attribute("href"))

In [114]:
driver.quit()

In [124]:
driver.find_element(By.XPATH, './/li[@cclass="fr-pb-0 ng-binding ng-scope"]')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@cclass="card-notification fr-callout fr-callout--boamp fr-py-2v fr-px-2w fr-px-md-3w fr-mb-4w ng-scope"]"}
  (Session info: chrome=137.0.7151.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010104dd14 cxxbridge1$str$ptr + 2735276
1   chromedriver                        0x0000000101045f88 cxxbridge1$str$ptr + 2703136
2   chromedriver                        0x0000000100b966f0 cxxbridge1$string$len + 90424
3   chromedriver                        0x0000000100bdd9e0 cxxbridge1$string$len + 381992
4   chromedriver                        0x0000000100c1f0c8 cxxbridge1$string$len + 650000
5   chromedriver                        0x0000000100bd1be8 cxxbridge1$string$len + 333360
6   chromedriver                        0x0000

In [143]:
print(len(liens))

60


In [90]:
len(driver.find_elements(By.XPATH, './/a[@class="fr-btn fr-my-1w ng-binding"]'))

10

In [83]:
driver.find_element(By.XPATH, './/div[@class="fr-container no-print"]/nav/ul/li[8]').text

'8'

In [84]:
len(driver.find_elements(By.XPATH, './/ul[@class="no-list-style ng-scope"]/li'))

10